In [ ]:
def comfy_install():
  %cd /content/
  import concurrent.futures
  from concurrent.futures import ThreadPoolExecutor, as_completed
  %env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:512
  %env TF_CPP_MIN_LOG_LEVEL=3
  !apt -y update -qq > /dev/null 2>&1
  !sudo apt-get install libjemalloc-dev > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbb.so.12 /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1
  %env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so
  !apt install -qq libunwind8-dev
  subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI"], cwd="/content/")
  %cd /content/ComfyUI
  !npm install -g localtunnel
  !pip install -q xformers==0.0.20 triton==2.0.0 -U pyngrok tomesd rembg[gpu] mediapipe==0.9.1.0 addict yapf fvcore omegaconf
  !pip install -r requirements.txt # --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117
  subprocess.run(["ngrok", "config", "add-authtoken", ngrok_token])
  with open('/content/ComfyUI/server.py', 'r') as file:
    lines = file.readlines()
  insert_index = None
  for i, line in enumerate(lines):
    if line.strip().startswith("if verbose:") and not lines[i+1].strip().startswith("from pyngrok import ngrok"):
      insert_index = i + 1
      break
  if insert_index is not None:
    ngrok_injection = [
        "            from pyngrok import ngrok\n",
        "            ngrok_tunnel = ngrok.connect(8188)\n",
        "            tunnel_url = ngrok_tunnel.public_url\n",
        "            print(\"\033[1m\033[32mссылка на нгрок:\033[0m\", tunnel_url)\n",
    ]
    lines.insert(insert_index, ngrok_injection[0])
    lines.insert(insert_index + 1, ngrok_injection[1])
    lines.insert(insert_index + 2, ngrok_injection[2])
    lines.insert(insert_index + 3, ngrok_injection[3])
    lines = lines[:insert_index + 4] + lines[insert_index + 6:]
    with open('/content/ComfyUI/server.py', 'w') as file:
      file.writelines(lines)
  wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "

  if мобильный:
    os.remove("/content/ComfyUI/web/style.css")
    !$wget https://huggingface.co/tryscat/dl/resolve/main/style.css -O /content/ComfyUI/web/style.css
    !$wget https://raw.githubusercontent.com/pythongosssss/ComfyUI-Custom-Scripts/main/js/touchEvents.js -P /content/ComfyUI/web/extensions
    with open("/content/ComfyUI/web/extensions/touchEvents.js", 'r+') as file:
      lines = file.readlines()
      for index, line in enumerate(lines):
        if 'import { app } from "/scripts/app.js";' in line:
          lines.insert(index + 1, 'function applyTouchAction(){const graphCanvas=document.querySelector("#graph-canvas");if(graphCanvas){graphCanvas.style.touchAction="pinch-zoom"}}function checkForCanvas(){const graphCanvas=document.querySelector("#graph-canvas");if(graphCanvas){applyTouchAction();clearInterval(interval)}}const interval=setInterval(checkForCanvas,100);const observer=new MutationObserver(checkForCanvas);observer.observe(document.body,{childList:true,subtree:true});\n')
          break
      file.seek(0)
      file.writelines(lines)
      file.truncate()
  # скачивание моделей
  dlmodels = [
      f"https://huggingface.co/2ch/sdxl/resolve/main/sd_xl_base_1.0.safetensors -P /content/ComfyUI/models/checkpoints/", # модель
      f"https://huggingface.co/2ch/sdxl/resolve/main/sd_xl_refiner_1.0.safetensors -P /content/ComfyUI/models/checkpoints/", # рефайнер
      #f"https://huggingface.co/2ch/sdxl/resolve/main/sd_xl_base_0.9.safetensors -P /content/ComfyUI/models/checkpoints/, # модель old
      #f"https://huggingface.co/2ch/sdxl/resolve/main/sd_xl_refiner_0.9.safetensors -P /content/ComfyUI/models/checkpoints/", # рефайнер old
      "https://huggingface.co/2ch/sdxl/resolve/main/sdxl_vae.safetensors -P /content/ComfyUI/models/vae/", # vae
      "https://civitai.com/api/download/models/21518 -P /content/ComfyUI/models/loras/", # лора ГигаЧед для примера
      # sdxl controlnet
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank128.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank128.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank128.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank128-metadata.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -P /content/ComfyUI/models/controlnet/",
      # Controlnet & T2I-Adapter
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P /content/ComfyUI/models/controlnet/",
      # T2I Styles Model
      "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P /content/ComfyUI/models/style_models/",
      "https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O /content/ComfyUI/models/clip_vision/clip_vit14.bin",
      # GLIGEN
      "https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P /content/ComfyUI/models/gligen/",
      # ESRGAN upscale models
      "https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P /content/ComfyUI/models/upscale_models/",
      "https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P /content/ComfyUI/models/upscale_models/",
      "https://huggingface.co/utnah/esrgan/resolve/main/4x_NMKD-Siax_200k.pth -P /content/ComfyUI/models/upscale_models/",
      "https://huggingface.co/utnah/esrgan/resolve/main/4x-UltraSharp.pth -P /content/ComfyUI/models/upscale_models/",
      "https://huggingface.co/utnah/esrgan/resolve/main/x1_ITF_SkinDiffDetail_Lite_v1.pth -P /content/ComfyUI/models/upscale_models/",
      # просмотр картинок
      "https://raw.githubusercontent.com/ailex000/ComfyUI-Extensions/main/image-gallery/imageGallery.js -P /content/ComfyUI/web/extensions/imageGallery",
    ]
  def download_models(dlmodel):
    !$wget $dlmodel
  if not test:
    with ThreadPoolExecutor(max_workers=len(dlmodels)) as executor:
      futures = [executor.submit(download_models, dlmodel) for dlmodel in dlmodels]
      for future in as_completed(futures):
          future.result()
  # дополнения     #"https://github.com/Zuellni/ComfyUI-Custom-Nodes Zuellni", - задолбал своим bitsandbytes
  #     "https://github.com/Davemane42/ComfyUI_Dave_CustomNode", не поддерживается
  custom_nodes = [
    "https://github.com/PR0LAPSE/workflow_menu",
    "https://github.com/bmad4ever/ComfyUI-Bmad-DirtyUndoRedo",
    "https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb",
    "https://github.com/LEv145/images-grid-comfy-plugin",
    "https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes",
    "https://github.com/BadCafeCode/masquerade-nodes-comfyui",
    "https://github.com/BlenderNeko/ComfyUI_Cutoff",
    "https://github.com/biegert/ComfyUI-CLIPSeg",
    "https://github.com/YinBailiang/MergeBlockWeighted_fo_ComfyUI",
    "https://github.com/Jcd1230/rembg-comfyui-node",
    "https://github.com/WASasquatch/was-node-suite-comfyui",
    "https://github.com/space-nuko/ComfyUI-OpenPose-Editor",
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
    "https://github.com/Fannovel16/comfyui_controlnet_aux",
    "https://github.com/ltdrdata/ComfyUI-Manager",
    "https://github.com/daxthin/facedetailer",
    "https://github.com/M1kep/ComfyLiterals",
  ]
  def run_custom_node(custom_node):
    node_parts = custom_node.split()
    url = node_parts[0]
    folder_name = node_parts[1] if len(node_parts) > 1 else ""
    command = ["git", "clone", url]
    if folder_name:
      command.append(folder_name)
    subprocess.run(command, cwd="/content/ComfyUI/custom_nodes")
  with concurrent.futures.ThreadPoolExecutor(max_workers=len(custom_nodes)) as executor:
    futures = [executor.submit(run_custom_node, custom_node) for custom_node in custom_nodes]
    concurrent.futures.wait(futures)
  if not test:
    clear_output()
  print("сейчас дополнения устанавливают свои зависимости...")
  subprocess.run(["pip", "install", "-r", "requirements.txt"], cwd="/content/ComfyUI/custom_nodes/comfyui_controlnet_aux")
  subprocess.run(["pip", "install", "-r", "requirements.txt"], cwd="/content/ComfyUI/custom_nodes/was-node-suite-comfyui")
  # subprocess.run(["pip", "install", "-U", "-r", "requirements.txt"], cwd="/content/ComfyUI/custom_nodes/Zuellni")
  subprocess.run(["pip", "install", "-r", "requirements.txt"], cwd="/content/ComfyUI/custom_nodes/ComfyUI-CLIPSeg")
  shutil.rmtree('/content/ComfyUI/custom_nodes/ComfyUI-CLIPSeg')
  subprocess.run(["pip", "uninstall", "bitsandbytes", "-y"])
  #with open('/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack/impact-pack.ini', 'r+') as file:
  #  text = file.read()
  #  text = text.replace('mmdet_skip = True', 'mmdet_skip = False')
  #  file.seek(0)
  #  file.write(text)
  #  file.truncate()

  impact_pack_ini = """
  [default]
  dependency_version = 11
  mmdet_skip = False
  sam_editor_cpu = False
  sam_editor_model = sam_vit_b_01ec64.pth
  """
  #os.makedirs(os.path.dirname("/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack/impact-pack.ini"), exist_ok=True)
  with open("/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack/impact-pack.ini", "w") as file:
      file.write(impact_pack_ini.strip())

  subprocess.run(["python", "install.py"], cwd="/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack")
  if os.path.exists("/content/ComfyUI/custom_nodes/was-node-suite-comfyui/was_suite_config.json"):
    import json
    with open('/content/ComfyUI/custom_nodes/was-node-suite-comfyui/was_suite_config.json', 'r') as file:
      data = json.load(file)
    data['ffmpeg_bin_path'] = '/usr/bin/ffmpeg'
    data['run_requirements'] = False
    data['suppress_uncomfy_warning']  = True
    data['show_startup_junk']  = False
    data['show_inspiration_quote']  = False # это хлеще цитат из библии...
    with open('/content/ComfyUI/custom_nodes/was-node-suite-comfyui/was_suite_config.json', 'w') as file:
      json.dump(data, file, indent=4)

  WAS_path = '/content/ComfyUI/custom_nodes/was-node-suite-comfyui/WAS_Node_Suite.py'
  replacement_text = 'show_quotes = False'
  with open(WAS_path, 'r') as file:
      lines = file.readlines()
  with open(WAS_path, 'w') as file:
      for line in lines:
          if 'show_quotes = True' in line:
              line = replacement_text + '\n'
          file.write(line)


def iframe_thread(port):
  while True:
    time.sleep(0.5)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', port))
    if result == 0:
      break
    sock.close()
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  with open('/content/ComfyUI/server.py', 'r') as server_py: content = server_py.read()
  if 'from pyngrok import ngrok' not in content:
    from pyngrok import ngrok
    ngrok_tunnel = ngrok.connect(8188)
    tunnel_url = ngrok_tunnel.public_url
    print("\033[1m\033[32mссылка на нгрок:\033[0m", tunnel_url)
  for line in p.stdout:
    print(line.decode(), end='')
    print("ip для вставки в localt:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))